# 0. import dependcies of what we will be using 


In [ ]:
import gymnasium as gym 
from stable_baselines3 import DQN 
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy  
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold 
from stable_baselines3.common.env_util import make_vec_env
import os 
import torch.nn as nn

# 1. Making The Enviornment 

In [2]:
env = gym.make("LunarLander-v2",render_mode = 'human') 

# 2. Designating a Path 

In [3]:
DQN_path = os.path.join('RL','Model')  

# 3. Callbacks 

In [4]:
stoptraining = StopTrainingOnRewardThreshold(reward_threshold=260,verbose=1) 
evalcallback = EvalCallback(env,callback_on_new_best = stoptraining,verbose=1)

# 4. Applying Wrappers 

In [5]:
env = make_vec_env('LunarLander-v2', n_envs=3, seed=None, start_index=0, 
                   monitor_dir=None, wrapper_class=None, env_kwargs=None, 
                   vec_env_cls=None, vec_env_kwargs=None)


# 5. Training Environment 

In [ ]:
model =DQN("MlpPolicy",env, learning_rate=0.0001, buffer_size=1000000, 
           learning_starts=50000, batch_size=32, tau=1.0, gamma=0.99, 
           train_freq=4, gradient_steps=1, replay_buffer_class=None, 
           replay_buffer_kwargs=None, optimize_memory_usage=False, 
           target_update_interval=10000, exploration_fraction=0.1, 
           exploration_initial_eps=1.0, exploration_final_eps=0.05, 
           max_grad_norm=10, stats_window_size=100, tensorboard_log=None, 
           policy_kwargs=None, verbose=1, seed=3, device='cuda', 
           _init_setup_model=True)
model.learn(total_timesteps=4000000, callback = evalcallback) 
model.save(DQN_path)

# 6.loading Model

In [15]:
model.save(DQN_path)
model = DQN.load(DQN_path) 


# 7. Evalutating Model 

In [14]:
evaluate_policy(model, env,n_eval_episodes = 10, render = True)

(204.75261719999997, 52.779680973137474)

# 8. Test out Algorithmn 

In [ ]:
obs = env.reset()
while True:
    action, states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()